# WAF Auto App - Complete Deployment

This notebook performs an end-to-end deployment:
1. ✅ Deploys the WAF Assessment dashboard to your workspace
2. ✅ Publishes the dashboard and gets the URL
3. ✅ Provides instructions to deploy the app with embedded dashboard

**Prerequisites:**
- Clone this repo to Databricks Repos: `https://github.com/AbhiDatabricks/Databricks-WAF-Light-Tooling`
- Checkout branch: `waf_auto_app`

**What gets deployed:**
- 📊 WAF Assessment Dashboard (Lakeview)
- 🚀 Instructions to deploy the waf-auto app


In [0]:
# Setup - Get context and credentials
import os, json, requests
from datetime import datetime

# Get user and context info
user = spark.sql("SELECT current_user()").collect()[0][0]
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
api_url = ctx.apiUrl().get()
token = ctx.apiToken().get()

print(f"✅ Connected to: {api_url}")
print(f"✅ User: {user}")


✅ Connected to: https://e2-demo-field-eng.cloud.databricks.com
✅ User: abhishekpratap.singh@databricks.com


In [0]:
# Get paths and setup
notebook_path = ctx.notebookPath().get()
base_path = "/".join(notebook_path.split("/")[:-2])  # Go up 2 levels (from waf-auto to repo root)
dashboard_folder = os.path.join(os.getcwd(), "..", "dashboards")
workspace_base_path = f"{base_path}/dashboards"

# Generate timestamp for unique naming
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

print(f"📂 Notebook path: {notebook_path}")
print(f"📂 Dashboard folder: {dashboard_folder}")
print(f"📂 Workspace base: {workspace_base_path}")
print(f"🕐 Timestamp: {timestamp}")
print()


📂 Notebook path: /Users/abhishekpratap.singh@databricks.com/Databricks-WAF-Light-Tooling-Nov24.13.25/waf-auto/(Clone) install
📂 Dashboard folder: /Workspace/Users/abhishekpratap.singh@databricks.com/Databricks-WAF-Light-Tooling-Nov24.13.25/waf-auto/../dashboards
📂 Workspace base: /Users/abhishekpratap.singh@databricks.com/Databricks-WAF-Light-Tooling-Nov24.13.25/dashboards
🕐 Timestamp: 20251124_0350



## Step 1: Deploy WAF Assessment Dashboard


In [0]:
# Deploy WAF Assessment Dashboard
dashboard_filename = "WAF_ASSESSMENTv2.1.lvdash.json"
dashboard_path = os.path.join(dashboard_folder, dashboard_filename)
base_dashboard_name = dashboard_filename.replace(".lvdash.json", "")
dashboard_name = f"{base_dashboard_name}_{timestamp}"

print(f"📊 Deploying: {base_dashboard_name}")
print(f"   Name: {dashboard_name}")
print(f"   File: {dashboard_path}")

# Load dashboard definition
with open(dashboard_path, "r", encoding="utf-8") as f:
    dashboard_def = json.load(f)

# Create or update dashboard
response = requests.post(
    url=f"{api_url}/api/2.0/lakeview/dashboards",
    headers={
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    },
    json={
        "display_name": dashboard_name,
        "parent_path": workspace_base_path,
        "serialized_dashboard": json.dumps(dashboard_def),
        "warehouse_id": None
    }
)

dashboard_id = None
dashboard_url = None

if response.status_code in [200, 201]:
    result = response.json()
    dashboard_id = result.get("dashboard_id", "")
    dashboard_path_result = result.get("path", "")
    dashboard_url = f"{api_url}/sql/dashboardsv3/{dashboard_id}"
    
    print(f"✅ Created dashboard: {dashboard_name}")
    print(f"   📊 Dashboard ID: {dashboard_id}")
    print(f"   🔗 Path: {dashboard_path_result}")
    print(f"   🌐 URL: {dashboard_url}")
elif response.status_code == 400:
    error_json = response.json()
    if error_json.get("error_code") == "RESOURCE_ALREADY_EXISTS":
        print(f"⚠️  Dashboard already exists, finding it...")
        
        # List dashboards
        list_response = requests.get(
            url=f"{api_url}/api/2.0/lakeview/dashboards",
            headers={"Authorization": f"Bearer {token}"}
        )
        
        if list_response.status_code == 200:
            dashboards = list_response.json().get("dashboards", [])
            existing = None
            
            for dash in dashboards:
                if dash.get("display_name") == dashboard_name:
                    existing = dash
                    break
            
            if existing:
                dashboard_id = existing["dashboard_id"]
                dashboard_url = f"{api_url}/sql/dashboardsv3/{dashboard_id}"
                
                # Update it
                update_response = requests.patch(
                    url=f"{api_url}/api/2.0/lakeview/dashboards/{dashboard_id}",
                    headers={
                        "Authorization": f"Bearer {token}",
                        "Content-Type": "application/json"
                    },
                    json={"serialized_dashboard": json.dumps(dashboard_def)}
                )
                
                if update_response.status_code == 200:
                    print(f"✅ Updated dashboard: {dashboard_name}")
                    print(f"   📊 Dashboard ID: {dashboard_id}")
                    print(f"   🌐 URL: {dashboard_url}")
                else:
                    print(f"❌ Failed to update: {update_response.status_code}")
            else:
                print(f"❌ Could not find existing dashboard")
    else:
        print(f"❌ Failed to create dashboard: {response.status_code}")
        print(f"Error: {response.text}")
else:
    print(f"❌ Failed to create dashboard: {response.status_code}")
    print(f"Error: {response.text}")

print()


📊 Deploying: WAF_ASSESSMENTv2.1
   Name: WAF_ASSESSMENTv2.1_20251124_0350
   File: /Workspace/Users/abhishekpratap.singh@databricks.com/Databricks-WAF-Light-Tooling-Nov24.13.25/waf-auto/../dashboards/WAF_ASSESSMENTv2.1.lvdash.json
✅ Created dashboard: WAF_ASSESSMENTv2.1_20251124_0350
   📊 Dashboard ID: 01f0c8e8ade512b39cc14a61ed891395
   🔗 Path: /Users/abhishekpratap.singh@databricks.com/Databricks-WAF-Light-Tooling-Nov24.13.25/dashboards/WAF_ASSESSMENTv2.1_20251124_0350.lvdash.json
   🌐 URL: https://e2-demo-field-eng.cloud.databricks.com/sql/dashboardsv3/01f0c8e8ade512b39cc14a61ed891395



In [0]:
import os

# Publish the dashboard
if dashboard_id:
    print(f"📤 Publishing dashboard...")
    publish_response = requests.post(
        url=f"{api_url}/api/2.0/lakeview/dashboards/{dashboard_id}/published",
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        json={"embed_credentials": True}
    )
    
    if publish_response.status_code in [200, 201]:
        print(f"✅ Dashboard published!")
        
        # Get org ID for embed URL
        org_id = os.environ.get("DATABRICKS_ORG_ID")
        dashboard_direct_url = f"{api_url}/sql/dashboardsv3/{dashboard_id}?o={org_id}"
        dashboard_embed_url = f"{api_url}/embed/dashboardsv3/{dashboard_id}?o={org_id}"
        
        print(f"\n{'='*70}")
        print(f"✅ DASHBOARD DEPLOYED SUCCESSFULLY")
        print(f"{'='*70}")
        print(f"📊 Dashboard Name: {dashboard_name}")
        print(f"🆔 Dashboard ID: {dashboard_id}")
        print(f"🌐 URL: {dashboard_direct_url}")
        print(f"📎 Embed URL: {dashboard_embed_url}")
        print(f"{'='*70}\n")
    else:
        print(f"⚠️  Failed to publish: {publish_response.status_code}")
        print(f"   Error: {publish_response.text}")
        dashboard_embed_url = f"{api_url}/embed/dashboardsv3/{dashboard_id}"
else:
    print(f"❌ No dashboard ID available - deployment failed")
    dashboard_embed_url = None

📤 Publishing dashboard...
✅ Dashboard published!

✅ DASHBOARD DEPLOYED SUCCESSFULLY
📊 Dashboard Name: WAF_ASSESSMENTv2.1_20251124_0350
🆔 Dashboard ID: 01f0c8e8ade512b39cc14a61ed891395
🌐 URL: https://e2-demo-field-eng.cloud.databricks.com/sql/dashboardsv3/01f0c8e8ade512b39cc14a61ed891395?o=None
📎 Embed URL: https://e2-demo-field-eng.cloud.databricks.com/embed/dashboardsv3/01f0c8e8ade512b39cc14a61ed891395?o=None



In [0]:
## Step 2: Deploy WAF Auto App

print("Now deploying the Databricks App that will embed this dashboard.")


Now deploying the Databricks App that will embed this dashboard.


In [0]:
# Deploy WAF Auto App
import time

app_name = "wafauto"
app_url = f"{api_url}/apps/{app_name}"

if dashboard_embed_url:
    print(f"{'='*70}")
    print(f"🚀 DEPLOYING WAF AUTO APP")
    print(f"{'='*70}\n")
    
    # Get repo path for the app - ensure it starts with /Workspace
    repo_path = base_path  # This is the repo root
    
    # Fix path format - if it starts with /Users/, prepend /Workspace
    if repo_path.startswith("/Users/") and not repo_path.startswith("/Workspace/Users/"):
        repo_path = f"/Workspace{repo_path}"
    elif repo_path.startswith("/Repos/") and not repo_path.startswith("/Workspace/Repos/"):
        repo_path = f"/Workspace{repo_path}"
    
    app_source_path = f"{repo_path}/waf-auto"
    
    print(f"📂 Repo path: {repo_path}")
    print(f"📂 App source path: {app_source_path}")
    print(f"📎 Dashboard embed URL: {dashboard_embed_url}\n")
    
    # Try to create the app (if it exists, we'll get 409 and skip to deployment)
    print(f"📱 Creating/Updating Databricks App...")
    create_payload = {
        "name": app_name,
        "description": "WAF Assessment Dashboard App - Embedded Lakeview Dashboard",
        "source_code_path": app_source_path
    }
    
    create_response = requests.post(
        url=f"{api_url}/api/2.0/apps",
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        json=create_payload
    )
    
    app_created_or_exists = False
    
    if create_response.status_code in [200, 201]:
        print(f"✅ App created: {app_name}")
        app_created_or_exists = True
        time.sleep(3)
    elif create_response.status_code == 409:
        print(f"ℹ️  App '{app_name}' already exists - will redeploy with latest code")
        app_created_or_exists = True
    else:
        print(f"❌ Failed to create app: {create_response.status_code}")
        print(f"   Error: {create_response.text}")
    
    # Deploy the app (works for both new and existing apps)
    if app_created_or_exists:
        print(f"\n🚀 Deploying app from source code...")
        deploy_payload = {
            "source_code_path": app_source_path,
            "mode": "SNAPSHOT"
        }
        
        deploy_response = requests.post(
            url=f"{api_url}/api/2.0/apps/{app_name}/deployments",
            headers={
                "Authorization": f"Bearer {token}",
                "Content-Type": "application/json"
            },
            json=deploy_payload
        )
        
        if deploy_response.status_code in [200, 201]:
            print(f"✅ Deployment initiated!")
            deploy_result = deploy_response.json()
            deployment_id = deploy_result.get("deployment_id", "unknown")
            print(f"   Deployment ID: {deployment_id}")
            
            # Wait for deployment to complete
            print(f"\n⏳ Waiting for deployment to complete...")
            print(f"   (Building Docker image, installing dependencies, starting app...)")
            time.sleep(15)  # Initial wait for build to start
            
            max_wait = 300  # 5 minutes max
            wait_time = 0
            deployment_complete = False
            last_state = None
            
            while wait_time < max_wait:
                # Check deployment status
                status_response = requests.get(
                    url=f"{api_url}/api/2.0/apps/{app_name}",
                    headers={"Authorization": f"Bearer {token}"}
                )
                
                if status_response.status_code == 200:
                    app_status = status_response.json()
                    compute_status = app_status.get("compute_status", {})
                    state = compute_status.get("state", "UNKNOWN")
                    
                    if state != last_state:
                        print(f"   📊 Status: {state}")
                        last_state = state
                    
                    if state == "ACTIVE":
                        deployment_complete = True
                        print(f"\n✅ Deployment completed successfully!")
                        break
                    elif state in ["FAILED", "TERMINATED"]:
                        print(f"\n❌ Deployment failed with state: {state}")
                        print(f"   Check logs at: {app_url}")
                        break
                
                time.sleep(20)
                wait_time += 20
            
            if deployment_complete:
                print(f"\n{'='*70}")
                print(f"🎉 DEPLOYMENT COMPLETE!")
                print(f"{'='*70}")
                print(f"📱 App Name: {app_name}")
                print(f"🌐 App URL: {app_url}")
                print(f"📊 Dashboard Embedded: Yes")
                print(f"🔄 Full-height iframe: Yes (latest code)")
                print(f"{'='*70}\n")
                print(f"👉 Click to open app: {app_url}")
                print(f"\n💡 The dashboard should now fill the entire page!")
            elif wait_time >= max_wait:
                print(f"\n⏱️  Deployment is still in progress (taking longer than expected)")
                print(f"   Status URL: {app_url}")
                print(f"   The app should be ready in a few more minutes")
                print(f"   Refresh the page to check status")
        else:
            print(f"❌ Failed to deploy: {deploy_response.status_code}")
            print(f"   Error: {deploy_response.text}")
            print(f"\n📋 Troubleshooting:")
            print(f"   1. Verify source code exists at: {app_source_path}")
            print(f"   2. Check app.yaml is present in source path")
            print(f"   3. Ensure requirements.txt is present")
            print(f"   4. Try manual deployment from: {app_url}")
else:
    print(f"❌ Cannot deploy app - dashboard embed URL not available")

🚀 DEPLOYING WAF AUTO APP

📂 Repo path: /Workspace/Users/abhishekpratap.singh@databricks.com/Databricks-WAF-Light-Tooling-Nov24.13.25
📂 App source path: /Workspace/Users/abhishekpratap.singh@databricks.com/Databricks-WAF-Light-Tooling-Nov24.13.25/waf-auto
📎 Dashboard embed URL: https://e2-demo-field-eng.cloud.databricks.com/embed/dashboardsv3/01f0c8e8ade512b39cc14a61ed891395?o=None

📱 Creating/Updating Databricks App...
ℹ️  App 'wafauto' already exists - will redeploy with latest code

🚀 Deploying app from source code...
✅ Deployment initiated!
   Deployment ID: 01f0c8e8b08a170c9a3355e18a7a44ec

⏳ Waiting for deployment to complete...
   (Building Docker image, installing dependencies, starting app...)
   📊 Status: ACTIVE

✅ Deployment completed successfully!

🎉 DEPLOYMENT COMPLETE!
📱 App Name: wafauto
🌐 App URL: https://e2-demo-field-eng.cloud.databricks.com/apps/wafauto
📊 Dashboard Embedded: Yes
🔄 Full-height iframe: Yes (latest code)

👉 Click to open app: https://e2-demo-field-eng.cl